# Recommendation System For Top 5 Movies

## Team 4 - Phase Four Project

### Authors:
1. Hellen Muanki
2. Jesicah Mutiso
3. Endalkachew Dessalegne
4. Brian Waweru
   

## 1. Project Overview

A new movie streaming startup, aims to offer a highly personalized viewing experience to differentiate itself in a crowded market. One of its core features is a movie recommendation system that helps users discover films they are likely to enjoy, based on their past preferences.

Accurate and relevant movie recommendations to users based on factors such as movie ratings and rating counts.



## 1.1. Business Understanding 
### 1.1.1 Business Problem 


### 1.1.2. Key Business Questions

## 2. Data Understanding

In [ ]:
#import necessary libraries
import pandas as pd


In [26]:
# Accessing the Database
df_movies = pd.read_csv("data/movies.csv")
df_movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [23]:
df_ratings = pd.read_csv("data/ratings.csv") 
df_ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [15]:
df_movies.shape

(9742, 3)

In [24]:
df_ratings.shape

(100836, 4)

In [20]:
df_ratings.userId.value_counts(sort=False)


64     517
128     33
192     22
256    174
320     20
      ... 
319     35
383     34
447     78
511     52
575     36
Name: userId, Length: 610, dtype: int64

## 3. EDA

## 4. Modeling

## 5. Evaluation